In [1]:
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd



'pd.set_option(\'display.max_rows\', 500)\npd.set_option(\'display.max_columns\', 500)\npd.set_option(\'display.width\', 1000)\nfrom tqdm.auto import tqdm\nimport itertools\n\n# ====================================================\n# Read & preprocess data and save it to disk\n# ====================================================\ndef read_preprocess_data():\n    train = pd.read_parquet(\'../input/amex-data-integer-dtypes-parquet-format/train.parquet\')\n    features = train.drop([\'customer_ID\', \'S_2\'], axis = 1).columns.to_list()\n    cat_features = [\n        "B_30",\n        "B_38",\n        "D_114",\n        "D_116",\n        "D_117",\n        "D_120",\n        "D_126",\n        "D_63",\n        "D_64",\n        "D_66",\n        "D_68",\n    ]\n    num_features = [col for col in features if col not in cat_features]\n    \n    # Train FE\n    print(\'Starting train feature extraction\')\n    train_num_agg = train.groupby("customer_ID")[num_features].agg([\'first\', \'mean\', \'

In [3]:
import joblib
import random
import time
import itertools
import scipy as sp
import numpy as np
import pandas as pd
from tqdm import tqdm

import lightgbm as lgb
warnings.filterwarnings('ignore')
from itertools import combinations

#from catboost import CatBoostClassifier
pd.set_option('display.max_columns', 500)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

class CFG:
    input_dir = '../input/amex-fe/'
    seed = 42
    n_folds = 5
    target = 'target'

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
""
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe_plus_plus.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe_plus_plus.parquet')
    return train, test

def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)



In [4]:
%%time
train = pd.read_parquet('Train_compressed.parquet')

In [5]:
train.head()

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,S_11_last,...,D_107_last,B_36_mean,B_36_std,B_36_min,B_36_max,B_36_last,B_37_mean,B_37_std,B_37_min,B_37_max,B_37_last,R_26_mean,R_26_std,R_26_min,R_26_max,R_26_last,R_27_mean,R_27_std,R_27_min,R_27_max,R_27_last,D_108_mean,D_108_std,D_108_min,D_108_max,D_108_last,D_109_mean,D_109_std,D_109_min,D_109_max,D_109_last,D_110_mean,D_110_std,D_110_min,D_110_max,D_110_last,D_111_mean,D_111_std,D_111_min,D_111_max,D_111_last,B_39_mean,B_39_std,B_39_min,B_39_max,B_39_last,D_112_mean,D_112_std,D_112_min,D_112_max,D_112_last,B_40_mean,B_40_std,B_40_min,B_40_max,B_40_last,S_27_mean,S_27_std,S_27_min,S_27_max,S_27_last,D_113_mean,D_113_std,D_113_min,D_113_max,D_113_last,D_115_mean,D_115_std,D_115_min,D_115_max,D_115_last,D_118_mean,D_118_std,D_118_min,D_118_max,D_118_last,D_119_mean,D_119_std,D_119_min,D_119_max,D_119_last,D_121_mean,D_121_std,D_121_min,D_121_max,D_121_last,D_122_mean,D_122_std,D_122_min,D_122_max,D_122_last,D_123_mean,D_123_std,D_123_min,D_123_max,D_123_last,D_124_mean,D_124_std,D_124_min,D_124_max,D_124_last,D_125_mean,D_125_std,D_125_min,D_125_max,D_125_last,D_127_mean,D_127_std,D_127_min,D_127_max,D_127_last,D_128_mean,D_128_std,D_128_min,D_128_max,D_128_last,D_129_mean,D_129_std,D_129_min,D_129_max,D_129_last,B_41_mean,B_41_std,B_41_min,B_41_max,B_41_last,B_42_mean,B_42_std,B_42_min,B_42_max,B_42_last,D_130_mean,D_130_std,D_130_min,D_130_max,D_130_last,D_131_mean,D_131_std,D_131_min,D_131_max,D_131_last,D_132_mean,D_132_std,D_132_min,D_132_max,D_132_last,D_133_mean,D_133_std,D_133_min,D_133_max,D_133_last,R_28_mean,R_28_std,R_28_min,R_28_max,R_28_last,D_134_mean,D_134_std,D_134_min,D_134_max,D_134_last,D_135_mean,D_135_std,D_135_min,D_135_max,D_135_last,D_136_mean,D_136_std,D_136_min,D_136_max,D_136_last,D_137_mean,D_137_std,D_137

In [7]:
train.dtypes

P_2_mean         float32
P_2_std          float32
P_2_min          float32
P_2_max          float32
P_2_last         float32
                  ...   
D_66_nunique       int64
D_68_count         int64
D_68_last       category
D_68_nunique       int64
target             int64
Length: 919, dtype: object

In [8]:
for col in train.columns:
    if(col.startswith('int')):
        train[col] = train[col].astype('int8')
    if(col.startswith('float')):
        train[col] = train[col].astype('float8')

In [13]:
import os
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

def train_and_evaluate(train):
    # Label encode categorical features
    models = []
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        #test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float8') | (train.dtypes == 'float8')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        #test[col + '_round2'] = test[col].round(2)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    params = {
        'objective': 'binary',
        'metric': "binary_logloss",
        'boosting': 'dart',
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.25,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40,
        #'device' : 'gpu'
        }
    # Create a numpy array to store test predictions
    #test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        del x_train,y_train
        gc.collect()
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        
        gc.collect()
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 9000,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 80,
            verbose_eval = 250,
            feval = lgb_amex_metric
            )
        # Save best model
        joblib.dump(model, f'lgbm_fold{fold}_seed{CFG.seed}.pkl')
        models.append(model)
        model.save_model(f'LGBM_model_{fold}.txt')
        # Predict validation
        val_pred = model.predict(x_val)
        # Add to out of folds array
        oof_predictions[val_ind] = val_pred
        # Predict the test set
        #test_pred = model.predict(test[features])
        #test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        del x_val,y_val
        print(f'Our fold {fold} CV score is {score}')
        del lgb_train, lgb_valid
        gc.collect()
    # Compute out of folds metric
    #score = amex_metric(train[CFG.target], oof_predictions)
    #print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    #oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    #oof_df.to_csv(f'oof_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    # Create a dataframe to store test prediction
    #test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    #test_df.to_csv(f'test_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    return models
seed_everything(CFG.seed)


In [ ]:
%%time
modelos = train_and_evaluate(train[:360000])

 
--------------------------------------------------
Training fold 0 with 918 features...
[LightGBM] [Info] Number of positive: 74696, number of negative: 213304
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.635195 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 220814
[LightGBM] [Info] Number of data points in the train set: 288000, number of used features: 917
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259361 -> initscore=-1.049292
[LightGBM] [Info] Start training from score -1.049292
[250]	training's binary_logloss: 0.416137	training's amex_metric: 0.772385	valid_1's binary_logloss: 0.419459	valid_1's amex_metric: 0.756186
[500]	training's binary_logloss: 0.337283	training's amex_metric: 0.780127	valid_1's binary_logloss: 0.342702	valid_1's amex_metric: 0.76051
[750]	training's binary_logloss: 0.275288	training's amex_metric: 0.789083	valid_1's binary_logloss: 0.283644	valid_1's amex_